In [1]:
import pandas as pd
import pulp
import numpy as np

In [2]:
time = 8
AC_max = 4250
AC_half = 2500
total_AC = 6
total_LTC = 3
total_STC = 2
LTC_cap = 7300
STC_cap = 2600

In [3]:
energy_params = {'AC_cool' : [[25, -0.65, 0.72], [25,     -0.7,   0]],
                 'AC_heat' : [[20, -0.65, 0.75], [250,    -0.95,  0]],
                 'STC'     : [[50, -0.85, 0.28], [25.002, -0.784, 0]],
                 'LTC'     : [[50, -0.85, 0.28], [25.002, -0.784, 0]]}

In [4]:
def energy_cons_func(e_prod, param):
    A, B, C = param[0], param[1], param[2]
    return ((A * (e_prod ** B)) + C) * e_prod

In [5]:
def cost(gas,elec):
    AC_cost = list()
    STC_cost = list()
    LTC_cost = list()
    for i in range(len(gas)):
        gas_rate = gas[i]
        elec_rate = elec[i]
        AC_gas = energy_cons_func(AC_max, energy_params['AC_cool'][0])
        AC_elec = energy_cons_func(AC_max,energy_params['AC_cool'][1])
        STC_gas = energy_cons_func(STC_cap, energy_params['STC'][0])
        STC_elec = energy_cons_func(STC_cap, energy_params['STC'][1])
        LTC_gas = energy_cons_func(LTC_cap, energy_params['LTC'][0])
        LTC_elec = energy_cons_func(LTC_cap, energy_params['LTC'][1])
        AC_cost.append(AC_gas*gas_rate + AC_elec*elec_rate)
        STC_cost.append(STC_gas*elec_rate + STC_elec*elec_rate)
        LTC_cost.append(LTC_gas*elec_rate + LTC_elec*elec_rate)
    
    return np.array(AC_cost), np.array(STC_cost), np.array(LTC_cost)

In [6]:
elec = np.array([0,49.64,49.64,49.64,49.64,49.64,49.64,49.64,49.64])
gas = np.array([0,10.83,10.83,10.83,10.83,10.83,10.83,10.83,10.83])

In [7]:
AC_cost, STC_cost, LTC_cost = cost(gas,elec)

In [8]:
demand = np.array([0,17495.95242,
10715.21031,
11420.41855,
11561.04044,
11707.86874,
11748.10859,
12243.31604,
12976.50566
])

In [9]:
AC_Run = pulp.LpVariable.dicts("AC_Run", range(time+1), lowBound=0,cat='Integer')
AC_On = pulp.LpVariable.dicts("AC_On", range(time+1), lowBound=0, cat='Integer')
AC_Off = pulp.LpVariable.dicts("AC_Off", range(time+1), lowBound=0, cat='Integer')
AC_Rest = pulp.LpVariable.dicts("AC_Rest", range(time+1), lowBound=0, cat='Integer')
AC_energy = pulp.LpVariable.dicts("AC_energy", range(time+1), lowBound=0, cat='Integer')

LTC_Run = pulp.LpVariable.dicts("LTC_Run", range(time+1), lowBound=0,cat='Integer')
LTC_On = pulp.LpVariable.dicts("LTC_On", range(time+1), lowBound=0, cat='Integer')
LTC_Off = pulp.LpVariable.dicts("LTC_Off", range(time+1), lowBound=0, cat='Integer')
LTC_Rest = pulp.LpVariable.dicts("LTC_Rest", range(time+1), lowBound=0, cat='Integer')
LTC_energy = pulp.LpVariable.dicts("LTC_energy", range(time+1), lowBound=0, cat='Integer')

STC_Run = pulp.LpVariable.dicts("STC_Run", range(time+1), lowBound=0,cat='Integer')
STC_On = pulp.LpVariable.dicts("STC_On", range(time+1), lowBound=0, cat='Integer')
STC_Off = pulp.LpVariable.dicts("STC_Off", range(time+1), lowBound=0, cat='Integer')
STC_Rest = pulp.LpVariable.dicts("STC_Rest", range(time+1), lowBound=0, cat='Integer')
STC_energy = pulp.LpVariable.dicts("STC_energy", range(time+1), lowBound=0, cat='Integer')

In [10]:
model = pulp.LpProblem("Cost_minimize_problem", pulp.LpMinimize)

In [11]:
model += pulp.lpSum(AC_Run[i+1]*AC_cost[i+1] + STC_Run[i+1]*STC_cost[i+1] + LTC_Run[i+1]*LTC_cost[i+1]
 + AC_On[i+1] + AC_Off[i+1] + STC_On[i+1] + STC_Off[i+1] + LTC_On[i+1] + LTC_Off[i+1] for i in range(time))

In [12]:
for t in range(time):
    model += AC_Run[t+1] == AC_Run[t] + AC_On[t+1] - AC_Off[t+1]
    model += STC_Run[t+1] == STC_Run[t] + STC_On[t+1] - STC_Off[t+1]
    model += LTC_Run[t+1] == LTC_Run[t] + LTC_On[t+1] - LTC_Off[t+1]

In [13]:
for t in range(time):
    model += AC_On[t+1] <= AC_Rest[t]
    model += STC_On[t+1] <= STC_Rest[t]
    model += LTC_On[t+1] <= LTC_Rest[t]

In [14]:
for t in range(time):
    model += AC_Off[t+1] <= AC_Run[t]
    model += STC_Off[t+1] <= STC_Run[t]
    model += LTC_Off[t+1] <= LTC_Run[t]

In [15]:
for t in range(time):
    model += AC_Rest[t+1] == AC_Rest[t] + AC_Off[t] - AC_On[t+1]
    model += STC_Rest[t+1] == STC_Rest[t] + STC_Off[t] - STC_On[t+1]
    model += LTC_Rest[t+1] == LTC_Rest[t] + LTC_Off[t] - LTC_On[t+1]

In [16]:
for t in range(time):
    model += AC_energy[t+1] == (AC_Run[t+1] - AC_On[t+1])*AC_max + AC_On[t+1]*AC_half
    model += STC_energy[t+1] == STC_Run[t+1]*STC_cap
    model += LTC_energy[t+1] == LTC_Run[t+1]*LTC_cap

In [17]:
for t in range(time):
    model += AC_energy[t+1] + STC_energy[t+1] + LTC_energy[t+1] >= demand[t+1]

In [18]:
model += AC_Off[0] == 0
model += AC_Run[0] == 0
model += AC_Rest[0] == total_AC
model += AC_On[0] == 0

model += STC_Off[0] == 0
model += STC_Run[0] == 0
model += STC_Rest[0] == total_STC
model += STC_On[0] == 0

model += LTC_Off[0] == 0
model += LTC_Run[0] == 0
model += LTC_Rest[0] == total_LTC
model += LTC_On[0] == 0

In [19]:
model.solve()

1

In [20]:
pulp.LpStatus[model.status]

'Optimal'

In [21]:
for t in range(time+1):
    print(AC_energy[t].varValue)

None
5000.0
11000.0
12750.0
12750.0
12750.0
12750.0
12750.0
8500.0


In [22]:
output = list()
for t in range(time+1):
    var_output = {'time':t,
                  'AC': AC_Run[t].varValue,
                  'STC': STC_Run[t].varValue,
                  'LTC': LTC_Run[t].varValue}
    output.append(var_output)
output = pd.DataFrame.from_records(output)

In [23]:
output

,time,AC,STC,LTC
0,0,0.0,0.0,0.0
1,1,2.0,2.0,1.0
2,2,3.0,0.0,0.0
3,3,3.0,0.0,0.0
4,4,3.0,0.0,0.0
5,5,3.0,0.0,0.0
6,6,3.0,0.0,0.0
7,7,3.0,0.0,0.0
8,8,2.0,2.0,0.0


In [24]:
AC_cost, STC_cost, LTC_cost

(array([    0.       , 53396.2376794, 53396.2376794, 53396.2376794,
        53396.2376794, 53396.2376794, 53396.2376794, 53396.2376794,
        53396.2376794]),
 array([    0.        , 50994.45441145, 50994.45441145, 50994.45441145,
        50994.45441145, 50994.45441145, 50994.45441145, 50994.45441145,
        50994.45441145]),
 array([     0.        , 119367.39039713, 119367.39039713, 119367.39039713,
        119367.39039713, 119367.39039713, 119367.39039713, 119367.39039713,
        119367.39039713]))